In [ ]:
import pandas as pd
import requests
from io import StringIO

### Problem-1:

You are given a SQL file link: https://drive.google.com/file/d/1WFt7B84LTHhMueoKmz8W-PRo7xXqmZf3/view?usp=share_link. Read the data by using the file and store it in a excel file. In this data, there are 3 tables named "invoices", "order_leads" and "sales_sql". So create 3 sheets to your excel file.

In [ ]:
import mysql.connector
connect = mysql.connector.connect(
    host='localhost',
    user='root',
    password='',
    database='temp_db'
)

invoices = pd.read_sql_query('SELECT * FROM invoices', connect)
order_leads = pd.read_sql_query('SELECT * FROM order_leads', connect)
sales_sql = pd.read_sql_query('SELECT * FROM sales_sql', connect)

with pd.ExcelWriter('supermarket.xlsx') as write:
    invoices.to_excel(write, sheet_name='invoices')
    order_leads.to_excel(write, sheet_name='order_leads')
    sales_sql.to_excel(write, sheet_name='sales_sql')

### Problem-2

Go to the site: https://rapidapi.com/wirefreethought/api/geodb-cities. From here, you have to grab the API and have to choose proper routes to get the cities of different countries. After getting the right API, hit that API and create a dataframe of all the cities that you can get by using the API. Then store the dataframe to a SQL. If you need to create an account or have to subscribe, then do that (it has free subscription but has some limitations. Use that free subscription and modify your accordingly to get all the data).  

In [ ]:
import requests

url = "https://wft-geo-db.p.rapidapi.com/v1/geo/adminDivisions"

headers = {
	"x-rapidapi-key": "bd4c6e1188mshdbbedd337739d43p139a1cjsndc90750d79f1",
	"x-rapidapi-host": "wft-geo-db.p.rapidapi.com"
}

response = requests.get(url, headers=headers)

print(response.json())

{'data': [{'id': 3515514, 'wikiDataId': 'Q4117373', 'name': 'Al Mamzar', 'country': 'United Arab Emirates', 'countryCode': 'AE', 'region': 'Emirate of Dubai', 'regionCode': 'DU', 'regionWdId': 'Q613', 'latitude': 25.309469444, 'longitude': 55.342811111, 'population': 0}, {'id': 3671889, 'wikiDataId': 'Q1262899', 'name': 'Dubai Internet City', 'country': 'United Arab Emirates', 'countryCode': 'AE', 'region': 'Emirate of Dubai', 'regionCode': 'DU', 'regionWdId': 'Q613', 'latitude': 25.095, 'longitude': 55.16, 'population': 24000}, {'id': 3850830, 'wikiDataId': 'Q102179103', 'name': 'Shumayliyah', 'country': 'United Arab Emirates', 'countryCode': 'AE', 'region': 'Emirate of Sharjah', 'regionCode': 'SH', 'regionWdId': 'Q188810', 'latitude': 25.33333, 'longitude': 56.33333, 'population': 0}, {'id': 3386644, 'wikiDataId': 'Q3694483', 'name': 'Ab Band District', 'country': 'Afghanistan', 'countryCode': 'AF', 'region': 'Ghazni', 'regionCode': 'GHA', 'regionWdId': 'Q180415', 'latitude': 32.983,

In [ ]:
data = response.json()['data']
links = response.json()['links']
metadata = response.json()['metadata']

In [ ]:
next_url = links[1]['href']
count = metadata['totalCount'] // 5

In [ ]:
import time
for i in range(count):
    url = 'https://wft-geo-db.p.rapidapi.com'
    page = url + next_url
    response = requests.get(page, headers=headers)
    print(response.json())
    links = response.json()['links']
    for item in links:
        if item['rel'] == 'next':
            next_url = item['href']

    data.extend(response.json()['data'])
    time.sleep(2)

{'data': [{'id': 3414199, 'wikiDataId': 'Q2380868', 'name': 'Bala Buluk District', 'country': 'Afghanistan', 'countryCode': 'AF', 'region': 'Farah', 'regionCode': 'FRA', 'regionWdId': 'Q180330', 'latitude': 32.69888889, 'longitude': 62.73, 'population': 0}, {'id': 3422672, 'wikiDataId': 'Q2215327', 'name': 'Balkhab', 'country': 'Afghanistan', 'countryCode': 'AF', 'region': 'Sar-e Pol', 'regionCode': 'SAR', 'regionWdId': 'Q182487', 'latitude': 36.22138889, 'longitude': 65.92777778, 'population': 0}, {'id': 3375638, 'wikiDataId': 'Q2572224', 'name': 'Bamyan', 'country': 'Afghanistan', 'countryCode': 'AF', 'region': 'Bamyan', 'regionCode': 'BAM', 'regionWdId': 'Q171382', 'latitude': 34.81666667, 'longitude': 67.81666667, 'population': 0}, {'id': 3338157, 'wikiDataId': 'Q3309222', 'name': 'Bangi', 'country': 'Afghanistan', 'countryCode': 'AF', 'region': 'Takhar', 'regionCode': 'TAK', 'regionWdId': 'Q186395', 'latitude': 36.613333333, 'longitude': 69.338055555, 'population': 0}, {'id': 3251

KeyboardInterrupt: 

In [ ]:
len(data)

1815

In [ ]:
cities = pd.DataFrame(data)
cities.shape

(1815, 11)

In [ ]:
import pymysql
from sqlalchemy import create_engine

engine = create_engine("mysql+pymysql://root:@localhost/temp_db")
cities.to_sql("cities",con=engine,if_exists='append')

### Problem 3:

Go to this url: https://www.flipkart.com/search?q=smartphones. This is the url to find phones in flipkart website. You have to extract the below things:
1. image url of the phone
2. name of the image
3. average ratings
4. total ratings
5. total reviews
6. discounted price
7. actual price

Extract all the phones which are available in this website. So you have to use the pagination concept. **Also after requesting every page through the url, please wait for a while (minimum 2-3 seconds), otherwise your IP address can be banned to access the flipkart website later.**

After collecting all the data, save that in a JSON file.

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import json
import time

def extract_phone_details(driver):
    phones = []

    # Find all product containers
    product_elements = driver.find_elements(By.XPATH, '//div[contains(@class, "tUxRFH")]')

    for product in product_elements:
        try:

            img = product.find_element(By.TAG_NAME, 'img')
            image_url = img.get_attribute('src')

            name = product.find_element(By.XPATH, './/div[contains(@class, "KzDlHZ")]').text  # Adjust based on the actual class
            ratings_reviews_element = product.find_element(By.XPATH, './/span[contains(@class, "Wphh3N")]')
            total_ratings = ratings_reviews_element.find_element(By.XPATH, './/span[1]').text.split()[0]  # Get the first span for ratings
            total_reviews = ratings_reviews_element.find_element(By.XPATH, './/span[3]').text.split()[0]  # Get the third span for reviews
            average_ratings = product.find_element(By.CLASS_NAME, 'XQDdHH').text

            discounted_price = product.find_element(By.XPATH, './/div[contains(@class, "Nx9bqj") and contains(@class, "_4b5DiR")]').text
            # actual_price = product.find_element(By.XPATH, './/div[contains(@class, "yRaY8j") and contains(@class, "ZYYwLA")]').text


            phone = {
                'image_url': image_url,
                'name': name,
                'average_ratings': average_ratings,
                'total_ratings': total_ratings,
                'total_reviews': total_reviews,
                'discounted_price': discounted_price,
                # 'actual_price': actual_price
            }

            phones.append(phone)

        except Exception as e:
            print(f"Error extracting details for a product: {e}")
            continue

    return phones

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

all_phones = []
base_url = 'https://www.flipkart.com/search?q=smartphones'

for page in range(1, 41):
    # Navigate to the page
    driver.get(f"{base_url}&page={page}")

    time.sleep(3)  # Wait for the page to load completely

    # Extract phone details from the current page
    phones_on_page = extract_phone_details(driver)
    all_phones.extend(phones_on_page)  # Add the current page's phones to the list

with open('flipkart_phones_without_actual_price.json', 'w') as json_file:
    json.dump(all_phones, json_file, indent=4)

driver.quit()
print(f"Extracted data for {len(all_phones)} phones and saved to 'flipkart_phones.json'.")